# Импортируем библиотеки

In [22]:
import datetime, time
import random
import pyads
import psycopg

## Тестовый коннект к серверу

In [2]:
import pyads
plc = pyads.Connection('192.168.1.177.1.1',801)
plc.open()
plc.close()

2022-04-25T20:15:44+0300 Info: Connected to 192.168.1.177
2022-04-25T20:15:44+0300 Info: connection closed by remote


In [3]:
def db_execute(query,data):
    print(f'OK -----   {query} + ',data)

## Класс для работы с базой данных

In [37]:
# тестовое подключение
TEST_DATA = (datetime.datetime.now(),datetime.datetime.now(),random.randint(0,100),'PLC.TEST_BOOL','admin')

Q_INSERT_IN_LOG = '''INSERT INTO process_flow_log
                    (
                        start_date, 
                        finish_date, 
                        event_duration, 
                        variable, 
                        user_name
                    )
                    VALUES (%s, %s, %s, %s, %s)'''

connection_info = "dbname=combiner_data user=postgres password=1122 host=localhost"
connection_to_db = psycopg.connect(connection_info)
connection_to_db.execute(Q_INSERT_IN_LOG,TEST_DATA)
connection_to_db.commit()
connection_to_db.close()

In [53]:
class Database():

    DB_NAME = 'combiner_data'
    DB_USER = 'postgres'
    DB_PASS = '1122'
    DB_HOST = 'localhost'
    DB_ALT = 'CSV'              # Альтернативная DB. На счучай сбоя в подключении.

    DB_CONNECTION_STR = f'dbname={DB_NAME} user={DB_USER} password={DB_PASS} host={DB_HOST}'

    def __init__(self):
        self.connected = False
        self.db_connection = self.connect()
        
    def connect(self):
        connection = psycopg.connect(self.DB_CONNECTION_STR)
        self.connected = True
        return connection

    def disconnect(self):
        self.db_connection.close()

    def send_query(self,query,data):
        if self.connected:
            self.db_connection.execute(query,data) #TRY EXCEПТ
    
    def commit(self):
        self.db_connection.commit()


db = Database()
for i in range(100):
    db.send_query(Q_INSERT_IN_LOG,TEST_DATA)
db.commit()
db.disconnect()


In [4]:
class Plc:
    def __init__(self,address):
        self.address = address
        self.connect()

    def connect(self):
        return (True,f'Connected to {self.address}')
    
    def read_by_name(self,path,name):
        return '200'

In [6]:
class Variable:

    def __init__(self,path,var_type,plc,
                name='',ctrl_type = 'state', test = False):
        self.path = path
        self.name = name if name else path
        self.var_type = var_type
        self.plc = plc
        self.test = test

    @property
    def value(self):
        if self.test:
            return self.test
        else: 
            return self.plc.read_by_name(self.path,self.var_type)# read from plc


In [7]:
class StateControler:

    QUERY = 'BASIC_QUERY'

    def __init__(self,var):
        self.var = var
        self.prev_state = self.var.value

    @property
    def change_detected(self):
        return self.var.value != self.prev_state
    
    def set_previous_state(self):
        self.prev_state = self.var.value

    def check_state(self):
        if self.change_detected:
            print('State is changed')
            self.execute_query()
            self.set_previous_state()

    def get_data(self):
        return 'Some DATA'

    def execute_query(self):
        db_execute(self.QUERY, self.get_data())


In [8]:
class SwitchControl(StateControler):

    QUERY = 'SWITCH_QUERY'

    last_change_time = ''

    @property
    def time_of_switch(self):
        return datetime.datetime.now()

    def get_data(self):
        name = self.var.name
        value = self.var.value
        timest = self.time_of_switch

        return (name,timest,value)

In [22]:
# функция которая начинает ивент

# функция которая заканчивает ивент

# проверка состояния Event-а
class SwitchEventControl(StateControler):
    
    QUERY = 'SWITCH EVENT QUERY'

    def __init__(self, var, condition=None):
        StateControler.__init__(self,var)
        self.condition = condition
        self.start_time = ''
        self.finish_time = ''

    def check_state(self):
        pass

    def _set_time(self):
        if self.start_time:
            self.finish_time = datetime.datetime.now()
        else:
            self.start_time = datetime.datetime.now()

    def _set_flag(self):
        self.flag = not self.flag

    def start_event(self):
            self._set_time()

    def finish_event(self):
            self._set_time()

    

In [5]:
plc = Plc('1')
plc.read_by_name(1,2)

'200'

In [ ]:
TEST_VAR = Variable('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL,Plc('1'), name='Switch 1',test=True)

In [19]:
# Подключаемся к контроллеру

plc = pyads.Connection('192.168.1.177.1.1',801)
plc.open()

2022-04-25T20:09:37+0300 Info: Connected to 192.168.1.177


In [14]:
import time

sec = SwitchEventControl(TEST_VAR)

sec.start_event()
print(sec.start_time)
time.sleep(5)
sec.finish_event()
print(sec.finish_time)

2022-04-25 19:49:16.356242
2022-04-25 19:49:21.361425


In [ ]:
var_c = Variable('MAIN.bSWITCH_1',pyads.PLCTYPE_BOOL,plc, name='Switch 1')
var_f = Variable('MAIN.bSWITCH_2',pyads.PLCTYPE_BOOL,plc, name='Switch 2')
var_d = Variable('MAIN.bSWITCH_3',pyads.PLCTYPE_BOOL,plc, name='Switch 3')
var_g = Variable('MAIN.bSWITCH_4',pyads.PLCTYPE_BOOL,plc, name='Switch 4')

s1_sc = SwitchControl(var_c)
s2_sc = SwitchControl(var_f)
s3_sc = SwitchControl(var_d)
s4_sc = SwitchControl(var_g)

while True:
    s1_sc.check_state()
    s2_sc.check_state()
    s3_sc.check_state()
    s4_sc.check_state()

plc.close()



In [20]:
plc.close()


2022-04-25T20:09:42+0300 Info: connection closed by remote


In [ ]:
a = True
b = 4 if not a else a